In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
import random
import os
import re
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from datetime import datetime

Using TensorFlow backend.


In [2]:
with open("./data/friends/all_scripts.txt") as scripts_fileobj:
    all_scripts = scripts_fileobj.read().strip().lower().decode('utf8').encode('ascii', errors='ignore')

In [3]:
def preprocess_text(text):
    lines = text.split("\n")
    dialogues = []
    dialogue = []
    for line in lines:
        if ":" in line:
            dialogues.append(" ".join(dialogue))
            dialogue = []
            dialogue.append(line)
        else:
            dialogue.append(line)
    
    text = "\n".join(dialogues)
    punctuations = set(re.findall(r"[^a-zA-Z0-9 ]",text))
    for punctuation in punctuations:
        if punctuation == "\n":
            text = text.replace(punctuation," NEWLINE ")
        else:
            text = text.replace(punctuation," "+punctuation+" ")
            
        
    return text

In [4]:
def remove_infrequent_tokens(tokens,min_count=10):
    word_count = {}
    new_tokens = []
    vocab = []
    for token in tokens:
        if token in word_count:
            word_count[token] +=1
        else:
            word_count[token]=1
    for token_word in tokens:
        if word_count[token_word]>min_count:
            new_tokens.append(token_word)
    return new_tokens

In [5]:
def get_latest_train(model_path):
    checkpoints = os.listdir(model_path)
    latest_checkpoint = ""
    highest_epoch = 0
    for checkpoint in checkpoints:
        current_epoch = int(re.findall("weights-improvement-(\d+)-(\d+\.\d+).+",checkpoint)[0][0])
        if highest_epoch < current_epoch:
            highest_epoch = current_epoch
            latest_checkpoint = checkpoint
    return latest_checkpoint,highest_epoch

In [6]:
all_scripts_cleaned = preprocess_text(all_scripts)
tokens = all_scripts_cleaned.split()


In [7]:
# print(len(tokens))
cleaned_tokens = remove_infrequent_tokens(tokens)
print("Number of cleaned tokens: {}".format(len(cleaned_tokens)))

Number of cleaned tokens: 1300961


In [8]:
vocab = list(set(cleaned_tokens))
print("Vocal length: {}".format(len(vocab)))

Vocal length: 3567


In [9]:
vocab_length = len(vocab)
characters2id = dict((c, i) for i, c in enumerate(vocab))
id2characters = dict((i, c) for i, c in enumerate(vocab))
section_length = 20
step = 2
sections = []
section_labels = []
for i in range(0,len(cleaned_tokens)-section_length,step):
    section_in = cleaned_tokens[i:i+section_length]
    section_out = cleaned_tokens[i+section_length]
    sections.append([characters2id[word] for word in section_in])
    section_labels.append(characters2id[section_out])

print("Number of training examples: {}".format(len(sections)))

Number of training examples: 650471


In [11]:
X = np.reshape(sections, (len(sections), section_length, 1))
print(X.shape)
X = X / float(vocab_length)
y = np.zeros((len(sections),vocab_length))
for i,section in enumerate(sections):
    y[i,section_labels[i]] = 1

(650471, 20, 1)


In [12]:
print(X.shape,y.shape)

(650471, 20, 1) (650471, 3567)


In [13]:
model = Sequential()
model.add(LSTM(1024, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(256))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam')

In [14]:
model_path = "./model/model3_friends/"
filepath=model_path+"weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [17]:
# Training
num_epochs = 10
checkpoint_filename,latest_epoch = get_latest_train(model_path)
train_filepath = model_path+checkpoint_filename
model.load_weights(train_filepath)
print("{} - Training resuming from epoch {}. Training for {} epochs.".format(datetime.now(),latest_epoch,num_epochs))
model.fit(X, y, epochs=latest_epoch+num_epochs, batch_size=128, callbacks=callbacks_list,initial_epoch=latest_epoch)
print("{} - Training finished".format(datetime.now()))

2017-11-11 12:20:47.437004 - Training resuming from epoch 60. Training for 10 epochs.
Epoch 61/70
650471/650471 [==============================] - 542s 834us/step - loss: 1.5879
Epoch 62/70
650471/650471 [==============================] - 527s 810us/step - loss: 1.5340
Epoch 63/70
650471/650471 [==============================] - 527s 810us/step - loss: 1.5037
Epoch 64/70
650471/650471 [==============================] - 539s 828us/step - loss: 1.4776
Epoch 65/70
650471/650471 [==============================] - 529s 813us/step - loss: 1.4491
Epoch 66/70
650471/650471 [==============================] - 528s 812us/step - loss: 1.4232
Epoch 67/70
650471/650471 [==============================] - 529s 813us/step - loss: 1.3944
Epoch 68/70
650471/650471 [==============================] - 528s 812us/step - loss: 1.3744
Epoch 69/70
650471/650471 [==============================] - 530s 815us/step - loss: 1.3469
Epoch 70/70
650471/650471 [==============================] - 532s 818us/step - loss: 1

In [19]:
# Testing
checkpoint_filename,latest_epoch = get_latest_train(model_path)
test_filepath = model_path+checkpoint_filename
print("Testing after {} epochs. Filename: {}".format(latest_epoch,test_filepath))
model.load_weights(test_filepath)
model.compile(loss='categorical_crossentropy', optimizer='adam')

start = np.random.randint(0, len(sections)-1)
pattern = sections[start]
print("Seed:", " ".join([id2characters[idx] for idx in pattern]).replace("NEWLINE","\n"))
predictions = []
# generate characters
for i in range(500):
#     print([id2characters[idx] for idx in pattern])    
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(vocab_length)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = id2characters[index]
    seq_in = [id2characters[value] for value in pattern]
    predictions.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print(" ".join(predictions).replace("NEWLINE","\n"))

Testing after 70 epochs. Filename: ./model/model3_friends/weights-improvement-70-1.3258.hdf5
Seed: : well , why don ' t we move this into the bedroom ? 
 phoebe : really ? 

chandler : oh . 
 do : yeah . 
 rachel : i you know . 
 phoebe : hey , . 
 ross : yep ? which guess to ) 
 ross : y you . 
 rachel : oh . well in . . . we whole here whole to with that ' that attention . ] the everybody do . 
 ross : yeah - yeah , ooh . i . ( to guy ) bye . 
 the scene bye chandler ' s ross . ross ross phoebe the having . ] 
 ross : ( , ) do s 
 [ on , judy the ? 
 
 : : , i is ' thought ! . i . ) the : ( know ) joey : the you . re for for your moving ! the was ? 
 rachel : no , i really . 
 joey : ( , ) and ? 
 ross : oh , ive my im ! . 
 rachel : oh . were , . was i ( m - i you rachel happy were and emma thinks i . . . 
 ross : ( . ) 
 : , i 
 ross : i . ' . want you were so something . 
 monica : no , we on a a ross . ( ross into ) which startled 
 be : ? 
 him : ( , other ) thank : hi said , on

In [ ]:
print(vocab)